In case you haven't, please execute the following cell **once per Workspace Session** to install all the necessary requirements.

In [ ]:
!bash run_me_first_on_floyd.sh

# Visual Question Answering: Part II

### LSTM Based Recurrent Model

This notebook is simply an execution of the code to build VQA model based on a `CNN + LSTM` based approach, I would highly encourage you to read the [full post here](https://sominwadhwa.github.io/blog/2018/01/01/de/)

<p align="center">
  <img src="https://cdn-images-1.medium.com/max/1600/1*YkxweTzgt4axMnaTH0nHHg.gif"/>
</p>

**Let's get all the necessary library imports**

In [1]:
import sys, warnings
warnings.filterwarnings("ignore")
from random import shuffle, sample
import pickle as pk
import gc

import numpy as np
import pandas as pd
import scipy.io
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Reshape
from keras.layers.recurrent import LSTM
from keras.layers import Merge
from keras.layers.merge import Concatenate
from keras.optimizers import SGD
from keras.utils import np_utils, generic_utils
from progressbar import Bar, ETA, Percentage, ProgressBar    
from keras.models import model_from_json
from sklearn.preprocessing import LabelEncoder
import spacy
#from spacy.en import English

from src.utils import *
from src.features import *

Using TensorFlow backend.


## Preprocessed Data

The open-source VQA dataset contains multiple open-ended questions about various images. All my experiments were performed with v1 of the dataset (though I've processed v2 of the dataset as well), which contains:

- 82,783 training images from COCO (common objects in context) dataset.
- 215,407 question-answer pairs for training images.
- 40,504 validation images to perform own testing.
- 121,512 question-answer pairs for validation images.

In [8]:
training_questions     = open("preprocessed/v1/ques_train.txt","rb").read().decode('utf8').splitlines()
training_questions_len = open("preprocessed/v1/ques_train_len.txt","rb").read().decode('utf8').splitlines()
answers_train          = open("preprocessed/v1/answer_train.txt","rb").read().decode('utf8').splitlines()
images_train           = open("preprocessed/v1/images_coco_id.txt","rb").read().decode('utf8').splitlines()
img_ids                = open('preprocessed/v1/coco_vgg_IDMap.txt').read().splitlines()
vgg_path               = "/floyd/input/vqa_data/coco/vgg_feats.mat"

Let's look at a couple of questions along with their answers. The first entry you see here is the **COCO Image ID** through with the image can be found at [http://cocodataset.org/#explore](http://cocodataset.org/#explore) by simply entering the image ID in the **search** column. 

In [3]:
sample(list(zip(images_train, training_questions, answers_train)), 5)

[('488792', 'Do you see any tall buildings?', 'yes'),
 ('336563', 'Is his left foot lifted?', 'yes'),
 ('508191',
  "What is the state on the motorcycle's license plate?",
  'california'),
 ('205826', 'Is there a lot of penguins in this picture?', 'yes'),
 ('61949', 'Are there any animals in the picture?', 'yes')]

In [6]:
nlp = spacy.load("en")
print ("Loaded WordVec")

Loaded WordVec


In [9]:
vgg_features = scipy.io.loadmat(vgg_path)
img_features = vgg_features['feats']
id_map = dict()
print ("Loaded VGG Weights")

Loaded VGG Weights


Load image features - `4096` sized vectors extracted from the last layer of a VGG network trained on the COCO Dataset.

In [10]:
gc.collect()

131

In [8]:
upper_lim = 1000 #Number of most frequently occurring answers in COCOVQA (Covering >80% of the total data)
training_questions, answers_train, images_train = freq_answers(training_questions, 
                                                               answers_train, images_train, upper_lim)
training_questions_len, training_questions, answers_train, images_train = (list(t) for t in zip(*sorted(zip(training_questions_len, 
                                                                                                          training_questions, answers_train, 
                                                                                                          images_train))))
print (len(training_questions), len(answers_train),len(images_train))

215407 215407 215407


In [9]:
lbl = LabelEncoder()
lbl.fit(answers_train)
nb_classes = len(list(lbl.classes_))
pk.dump(lbl, open('preprocessed/v1/label_encoder_lstm.sav','wb'))

### Defining the Network Architecture

In [10]:
batch_size               =      256
img_dim                  =     4096
word2vec_dim             =      300
#max_len                 =       30 # Required only when using Fixed-Length Padding

num_hidden_nodes_mlp     =     1024
num_hidden_nodes_lstm    =      512
num_layers_mlp           =        3
num_layers_lstm          =        3
dropout                  =       0.5
activation_mlp           =     'tanh'


`num_epochs`: Set to the number of epochs you'd wish to run the network for.

`log_interval`: This parameter sets the epoch interval after which a copy of the model weights will be saved.

In [12]:
# Change the following based on your usage, THESE WILL DIRECTLY AFFECT THE DURATION OF NETWORK TRAINING
num_epochs               =         300 
log_interval             =         15 

In [16]:
for ids in img_ids:
    id_split = ids.split()
    id_map[id_split[0]] = int(id_split[1])

In [14]:
image_model = Sequential()
image_model.add(Reshape(input_shape = (img_dim,), target_shape=(img_dim,)))
image_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 4096)              0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [15]:
language_model = Sequential()
language_model.add(LSTM(output_dim=num_hidden_nodes_lstm, 
                        return_sequences=True, input_shape=(None, word2vec_dim)))

for i in range(num_layers_lstm-2):
    language_model.add(LSTM(output_dim=num_hidden_nodes_lstm, return_sequences=True))
language_model.add(LSTM(output_dim=num_hidden_nodes_lstm, return_sequences=False))

language_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 512)         1665024   
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 512)         2099200   
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
Total params: 5,863,424
Trainable params: 5,863,424
Non-trainable params: 0
_________________________________________________________________


In [16]:
model = Sequential()
model.add(Merge([language_model, image_model], mode='concat', concat_axis=1))
for i in range(num_layers_mlp):
    model.add(Dense(num_hidden_nodes_mlp, init='uniform'))
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))
model.add(Dense(upper_lim))
model.add(Activation("softmax"))

In [17]:
model_dump = model.to_json()
open('lstm_structure'  + '.json', 'w').write(model_dump)

5825

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_1 (Merge)              (None, 4608)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              4719616   
_________________________________________________________________
activation_1 (Activation)    (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
activation_2 (Activation)    (None, 1024)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
__________

**And we're good to go!**

In [ ]:
for k in range(num_epochs):
    progbar = generic_utils.Progbar(len(training_questions))
    for ques_batch, ans_batch, im_batch in zip(grouped(training_questions, batch_size, 
                                                       fillvalue=training_questions[-1]), 
                                               grouped(answers_train, batch_size, 
                                                       fillvalue=answers_train[-1]), 
                                               grouped(images_train, batch_size, fillvalue=images_train[-1])):
        timestep = len(nlp(ques_batch[-1]))
        X_ques_batch = get_questions_tensor_timeseries(ques_batch, nlp, timestep)
        #print (X_ques_batch.shape)
        X_img_batch = get_images_matrix(im_batch, id_map, img_features)
        Y_batch = get_answers_sum(ans_batch, lbl)
        loss = model.train_on_batch([X_ques_batch, X_img_batch], Y_batch)
        progbar.add(batch_size, values=[('train loss', loss)])
    if k%log_interval == 0:
        model.save_weights("weights/LSTM" + "_epoch_{:02d}.hdf5".format(k))
model.save_weights("weights/LSTM" + "_epoch_{:02d}.hdf5".format(k))

 72448/215407 [=========>....................] - ETA: 1:42 - train loss: 2.4110

# Let's evaluate our model!

We're going to evalute our model on the validation set provided by the **VQA Dataset** which I've already preprocessed much like our training datasets. Refer to [VQA Evaluation](http://visualqa.org/evaluation.html).

While I have evaluated my pre-trained models over here, you might like to change the paths in order to evaluate your own models. This can be easily done in the following way -

1. Add `model_from_json(open('lstm_structure.json').read())` (instead of loading model structure from my dataset, use the one you just created.
2. Modify -> `model.load_weights("weights/<weights_file>")` (instead of loading weights from my pretrained models, use your own set stored under `weights` directory.

By default, we're going to load the weights & the model created at the beginning of your training loop (for testing purposes).

In [11]:
model = model_from_json(open('lstm_structure.json').read()) #fully trained model & weights present at /floyd/input/vqa_data/weights/
# In case you wish to evaluate the model you just trained, uncomment the following line of code & comment out the subsequent one -
#model.load_weights('weights/LSTM_epoch_00.hdf5')
model.load_weights('/floyd/input/vqa_data/weights/LSTM_epoch_45.hdf5')
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print ("Model Loaded with Weights")
model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Model Loaded with Weights
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_1 (Merge)              (None, 4608)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              4719616   
_________________________________________________________________
activation_1 (Activation)    (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
activation_2 (Activation)    (None, 

**Loading the validation preprocessed data**

In [12]:
val_imgs = open('preprocessed/v1/val_images_coco_id.txt','rb').read().decode('utf-8').splitlines()
val_ques = open('preprocessed/v1/ques_val.txt','rb').read().decode('utf-8').splitlines()
val_ans  = open('preprocessed/v1/answer_val.txt','rb').read().decode('utf-8').splitlines()

**Replace location of LabelEncoder to your own, otherwise this may affect accuracy.** To do so, simply change the `file_path` to `preprocessed/v1/label_encoder_<type>.sav`. 

In [13]:
label_encoder = pk.load(open('preprocessed/v1/label_encoder_lstm.sav','rb'))

In [14]:
y_pred = []
batch_size = 128 

#print ("Word2Vec Loaded!")

widgets = ['Evaluating ', Percentage(), ' ', Bar(marker='#',left='[',right=']'), ' ', ETA()]
pbar = ProgressBar(widgets=widgets)

In [17]:
for qu_batch,an_batch,im_batch in pbar(zip(grouped(val_ques, batch_size, 
                                                   fillvalue=val_ques[0]), 
                                           grouped(val_ans, batch_size, 
                                                   fillvalue=val_ans[0]), 
                                           grouped(val_imgs, batch_size, 
                                                   fillvalue=val_imgs[0]))):
    timesteps = len(nlp(qu_batch[-1]))
    X_ques_batch = get_questions_tensor_timeseries(qu_batch, nlp, timesteps)
    X_i_batch = get_images_matrix(im_batch, id_map, img_features)
    X_batch = [X_ques_batch, X_i_batch]
    y_predict = model.predict_classes(X_batch, verbose=0)
    y_pred.extend(label_encoder.inverse_transform(y_predict))

Evaluating N/A% [#                                             ] Time:  0:11:40


In [35]:
correct_val = 0.0
total = 0
f1 = open('res.txt','w')

for pred, truth, ques, img in zip(y_pred, val_ans, val_ques, val_imgs):
    t_count = 0
    for _truth in truth.split(';'):
        if pred == truth:
            t_count += 1 
    if t_count >=1:
        correct_val +=1
    else:
        correct_val += float(t_count)/3

    total +=1

    try:
        f1.write(str(ques))
        f1.write('\n')
        f1.write(str(img))
        f1.write('\n')
        f1.write(str(pred))
        f1.write('\n')
        f1.write(str(truth))
        f1.write('\n')
        f1.write('\n')
    except:
        pass

print ("Accuracy: ", round((correct_val/total)*100,2))
#f1.write('Final Accuracy is ' + str(round(correct_val/total),2))
f1.close()

Accuracy:  54.79


**The accuracy achieved here is quite close to what the authors of [VQA Paper](https://arxiv.org/pdf/1505.00468.pdf) have reported in their LSTM based approach.**

There you go, all set to participate in the next VQA Challenge!

If you do, however, would like to try out these models on your own custom images do checkout **`src/test.py`** with an image and a characterstic question.